## Example XRD data analysis

In this notebook we'll demo an example analysis using background-subtracted XRD data. The analysis consists of splitting some background-subtracted XRD data into segments containing one or more peaks, and then fitting a model of some linear background and some pseudo-Voigt peaks to each segment found.

In [31]:
# An IPython Magic to automatically reload any of this notebook's dependencies
# when they're edited (preventing you from needing to reload the kernel)
%load_ext autoreload
%autoreload 3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
# imports
import sys, pathlib, glob
import pandas as pd
import matplotlib.pyplot as plt

This notebook uses some pre-written analysis and plotting code stored in the "`src/xrd_analysis`" folder of the repository. Feel free to edit it if you'd like to make improvements.

In [33]:
# imports from the local code
repo_root_dir = pathlib.Path().resolve().parent

xrd_analysis_code_path = str(repo_root_dir / "src" / "xrd_analysis")
if xrd_analysis_code_path not in sys.path:
    sys.path.append(xrd_analysis_code_path)
from utils import get_raw_file_name
from analysis import get_peak_segments, get_segment_peak_fit
from plotting import plot_peak_segments, plot_segment_peak_fit

### Read the background-subtracted XRD data into a Pandas DataFrame

In [34]:
xrd_data_dir = repo_root_dir.parent / "xrd_example_files"

xrd_data_file_paths = glob.glob(f"{xrd_data_dir}/*.csv")

In [35]:
# Use only the first file as an example to show off the method
test_file_index = 0
xrd_data_file = xrd_data_file_paths[test_file_index]

In [36]:
print(get_raw_file_name(xrd_data_file))

ML_Wallace_04122019_1_NN_1_ZrP_CrucibleRetest_5-60_4min_BkgSubtracted.raw



In [37]:
xrd_df = pd.read_csv(xrd_data_file, skiprows=2, names=["angle", "counts"])

### Find segments in the data containing one or more peaks

In [38]:
peak_segments = get_peak_segments(xrd_df)

In [39]:
plot_peak_segments(xrd_df, peak_segments)

### Fit a background+peak(s) model to each segment independently

In [40]:
for i_seg, peak_seg in enumerate(peak_segments):
    # Pull out the part of the dataframe that's within the segment
    seg_df = xrd_df[
        ((xrd_df["angle"] >= peak_seg["min"]) & (xrd_df["angle"] <= peak_seg["max"]))
    ]
    fit_result = get_segment_peak_fit(xrd_df, peak_seg)
    plot_segment_peak_fit(seg_df, fit_result)